In [10]:
# Enhanced CSV exploration without pandas
import os
import csv
from collections import Counter, defaultdict

def comprehensive_csv_analysis(file_path, sample_size=1000):
    """Comprehensive CSV analysis without pandas"""
    
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return None
    
    print(f"Analyzing: {os.path.basename(file_path)}")
    print("="*60)
    
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        
        # Get headers
        headers = next(reader)
        num_cols = len(headers)
        print(f"Columns ({num_cols}): {headers}")
        
        # Read sample data
        rows = []
        for i, row in enumerate(reader):
            if i >= sample_size:
                break
            rows.append(row)
        
        total_rows = len(rows)
        print(f"Sample size: {total_rows} rows")
        
        # Basic statistics per column
        print("\nColumn Analysis:")
        column_stats = {}
        
        for col_idx, col_name in enumerate(headers):
            values = []
            missing_count = 0
            
            for row in rows:
                if col_idx < len(row) and row[col_idx].strip():
                    values.append(row[col_idx].strip())
                else:
                    missing_count += 1
            
            # Determine column type and statistics
            non_missing = len(values)
            missing_pct = (missing_count / total_rows) * 100
            
            # Try to detect data type
            numeric_count = 0
            date_like_count = 0
            unique_values = set(values)
            
            for val in values[:100]:  # Sample first 100 values
                # Check if numeric
                try:
                    float(val.replace(',', '').replace('$', ''))
                    numeric_count += 1
                except:
                    pass
                
                # Check if date-like
                if any(sep in val for sep in ['-', '/', ' ']) and any(char.isdigit() for char in val):
                    date_like_count += 1
            
            # Classify column type
            if numeric_count > len(values) * 0.8:
                col_type = "Numeric"
            elif date_like_count > len(values) * 0.5:
                col_type = "Date-like"
            elif len(unique_values) < non_missing * 0.1:
                col_type = "Categorical (Low cardinality)"
            elif len(unique_values) > non_missing * 0.9:
                col_type = "Text (High cardinality/ID)"
            else:
                col_type = "Text (Medium cardinality)"
            
            column_stats[col_name] = {
                'type': col_type,
                'missing_count': missing_count,
                'missing_pct': missing_pct,
                'unique_values': len(unique_values),
                'sample_values': list(unique_values)[:5]
            }
            
            print(f"  {col_name}:")
            print(f"    Type: {col_type}")
            print(f"    Missing: {missing_count} ({missing_pct:.1f}%)")
            print(f"    Unique values: {len(unique_values)}")
            if col_type.startswith("Categorical") and len(unique_values) <= 10:
                value_counts = Counter(values)
                print(f"    Top values: {dict(value_counts.most_common(3))}")
            elif len(unique_values) <= 5:
                print(f"    All values: {list(unique_values)}")
            else:
                print(f"    Sample values: {list(unique_values)[:3]}")
        
        # Business column identification
        print("\nBusiness Column Identification:")
        business_patterns = {
            'ID columns': ['id', 'key', 'identifier', 'code'],
            'Price/Amount': ['price', 'cost', 'amount', 'value', 'total', 'revenue'],
            'Date/Time': ['date', 'time', 'created', 'updated', 'timestamp'],
            'Category': ['category', 'type', 'class', 'group', 'segment'],
            'Rating/Score': ['rating', 'score', 'stars', 'review'],
            'Location': ['city', 'state', 'country', 'address', 'location', 'zip'],
            'Customer': ['customer', 'user', 'client', 'buyer'],
            'Product': ['product', 'item', 'sku', 'goods']
        }
        
        identified_cols = defaultdict(list)
        for col in headers:
            col_lower = col.lower()
            for pattern_type, keywords in business_patterns.items():
                if any(keyword in col_lower for keyword in keywords):
                    identified_cols[pattern_type].append(col)
        
        for pattern_type, cols in identified_cols.items():
            if cols:
                print(f"  {pattern_type}: {cols}")
        
        # Data quality assessment
        print("\nData Quality Assessment:")
        high_missing = [col for col, stats in column_stats.items() if stats['missing_pct'] > 50]
        moderate_missing = [col for col, stats in column_stats.items() if 10 < stats['missing_pct'] <= 50]
        
        overall_completeness = 100 - (sum(stats['missing_count'] for stats in column_stats.values()) / (total_rows * num_cols) * 100)
        
        print(f"  Overall completeness: {overall_completeness:.1f}%")
        if high_missing:
            print(f"  High missing (>50%): {high_missing}")
        if moderate_missing:
            print(f"  Moderate missing (10-50%): {moderate_missing}")
        
        # Sample rows
        print("\nSample Data (first 3 rows):")
        for i, row in enumerate(rows[:3]):
            print(f"  Row {i+1}: {dict(zip(headers, row))}")
        
        return {
            'filename': os.path.basename(file_path),
            'shape': (total_rows, num_cols),
            'columns': headers,
            'column_stats': column_stats,
            'business_columns': dict(identified_cols),
            'completeness': overall_completeness,
            'quality_issues': {
                'high_missing': high_missing,
                'moderate_missing': moderate_missing
            }
        }

# Analyze all your datasets
dataset_analyses = {}

# List your available datasets
datasets_to_analyze = [
    "/app/data/raw/kaggle_datasets/brazilian-ecommerce/olist_customers_dataset.csv",
    "/app/data/raw/kaggle_datasets/brazilian-ecommerce/olist_products_dataset.csv",
    "/app/data/raw/kaggle_datasets/brazilian-ecommerce/olist_orders_dataset.csv",
    "/app/data/raw/kaggle_datasets/brazilian-ecommerce/olist_order_items_dataset.csv",
    "/app/data/raw/kaggle_datasets/brazilian-ecommerce/olist_order_payments_dataset.csv",
    "/app/data/raw/kaggle_datasets/brazilian-ecommerce/olist_order_reviews_dataset.csv"
]

print("COMPREHENSIVE DATA EXPLORATION REPORT")
print("="*80)

for dataset_path in datasets_to_analyze:
    if os.path.exists(dataset_path):
        analysis = comprehensive_csv_analysis(dataset_path, sample_size=1000)
        if analysis:
            dataset_analyses[analysis['filename']] = analysis
        print("\n" + "="*80 + "\n")
    else:
        print(f"Dataset not found: {dataset_path}")

# Cross-dataset analysis
if dataset_analyses:
    print("CROSS-DATASET SUMMARY")
    print("="*50)
    
    print("Dataset Overview:")
    for name, analysis in dataset_analyses.items():
        print(f"  {name}: {analysis['shape']} - {analysis['completeness']:.1f}% complete")
    
    # Find common columns
    all_columns = {}
    for name, analysis in dataset_analyses.items():
        for col in analysis['columns']:
            if col not in all_columns:
                all_columns[col] = []
            all_columns[col].append(name)
    
    common_cols = {col: files for col, files in all_columns.items() if len(files) > 1}
    if common_cols:
        print("\nCommon columns across datasets:")
        for col, files in common_cols.items():
            print(f"  {col}: {files}")
    
    # Business use case assessment
    print("\nBusiness Use Case Assessment:")
    
    use_cases = {
        'Sales Forecasting': ['date', 'sales', 'amount', 'price', 'quantity', 'order'],
        'Customer Analysis': ['customer', 'user', 'buyer', 'purchase', 'order'],
        'Product Analytics': ['product', 'item', 'category', 'price', 'rating'],
        'Transaction Analysis': ['order', 'payment', 'amount', 'date', 'customer']
    }
    
    for use_case, required_keywords in use_cases.items():
        suitable_datasets = []
        for name, analysis in dataset_analyses.items():
            col_text = ' '.join(analysis['columns']).lower()
            matches = sum(1 for keyword in required_keywords if keyword in col_text)
            if matches >= 2:  # At least 2 matching keywords
                suitable_datasets.append((name, matches))
        
        print(f"\n{use_case}:")
        if suitable_datasets:
            for dataset, match_count in sorted(suitable_datasets, key=lambda x: x[1], reverse=True):
                print(f"  {dataset}: {match_count}/{len(required_keywords)} keyword matches")
        else:
            print("  No suitable datasets found")

print("\nNext Steps Recommendations:")
print("1. Focus on datasets with >80% completeness for initial development")
print("2. Plan data joining strategy using common columns")
print("3. Prioritize use cases based on data availability")
print("4. Design cleaning pipeline for identified quality issues")

COMPREHENSIVE DATA EXPLORATION REPORT
Analyzing: olist_customers_dataset.csv
Columns (5): ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
Sample size: 1000 rows

Column Analysis:
  customer_id:
    Type: Text (High cardinality/ID)
    Missing: 0 (0.0%)
    Unique values: 1000
    Sample values: ['36b735afba701eeba82799b7d22161b2', '57934d36535621517dc2b60295108836', 'db2cf3b3feeca251d6c3c93bf5f1b75a']
  customer_unique_id:
    Type: Text (High cardinality/ID)
    Missing: 0 (0.0%)
    Unique values: 999
    Sample values: ['d647ac91ff870deee189d7cd0cbf2380', '4f6d635ff2fd4e30ff5369a7b943eb22', '8c2a5c47e0c02478c440038904724cd5']
  customer_zip_code_prefix:
    Type: Text (High cardinality/ID)
    Missing: 0 (0.0%)
    Unique values: 935
    Sample values: ['65075', '04427', '30380']
  customer_city:
    Type: Text (Medium cardinality)
    Missing: 0 (0.0%)
    Unique values: 379
    Sample values: ['ibia', 'ibatiba', 'valenca']
  cus